In [2]:

import os
from typing import List
from langchain.schema import Document
from nltk.tokenize import sent_tokenize

import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/raluca/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:


def extract_poetic_spans(poem_text: str, max_lines: int = 4, max_tokens: int = 200) -> List[str]:
    """
    Extract poetic spans: chunks of 2–4 lines with semantic and formal coherence.
    Falls back to sentences if poem is prose-like.
    """
    lines = [line.strip() for line in poem_text.splitlines() if line.strip()]
    spans = []
    i = 0

    while i < len(lines):
        span_lines = lines[i:i + max_lines]
        span_text = "\n".join(span_lines)

        # Try to find semantic closure
        if not span_text.strip().endswith(('.', '!', '?', '”', '…')) and (i + max_lines < len(lines)):
            span_lines = lines[i:i + max_lines + 1]
            span_text = "\n".join(span_lines)

        # Token length filter (optional)
        if len(span_text.split()) <= max_tokens:
            spans.append(span_text)

        i += max_lines  # Can adjust for overlap if desired

    return spans


def process_poetry_corpus(directory_path: str) -> List[Document]:
    """
    Process all .txt poetry files in a directory into LangChain Documents.
    """
    all_documents = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()

            spans = extract_poetic_spans(text)
            for span in spans:
                doc = Document(
                    page_content=span,
                    metadata={"source": filename}
                )
                all_documents.append(doc)

    return all_documents


In [5]:

if __name__ == "__main__":
    poetry_dir = "parallel_universes_dna_corpus"
    poetic_documents = process_poetry_corpus(poetry_dir)

    print(f"Extracted {len(poetic_documents)} poetic spans from {poetry_dir}")
    print("Sample document:")
    print(poetic_documents[0].page_content)


Extracted 1446 poetic spans from parallel_universes_dna_corpus
Sample document:
In the dark of a cave, fear
Visits her. She who remembers
How to be woken out of the sleeping
Body of another.


In [6]:
poetic_documents[0]

Document(metadata={'source': 'allan_popa_eve_from_filipino_trans_jose_perez_beduya_jose_edmundo_ocampo_reyes_and_marc_gaba_poem_6.txt'}, page_content='In the dark of a cave, fear\nVisits her. She who remembers\nHow to be woken out of the sleeping\nBody of another.')

In [7]:

poetic_documents[1]

Document(metadata={'source': 'allan_popa_eve_from_filipino_trans_jose_perez_beduya_jose_edmundo_ocampo_reyes_and_marc_gaba_poem_6.txt'}, page_content='She will pluck out from herself the loss\nAs the exact punishment for her sin.\nWrapped in the hide of animals,\nHer eldest, borrowing its heat.')

In [8]:
poetic_documents[36]

Document(metadata={'source': 'alejandro_albarrán_cowboy_from_spanish_trans_rachel_galvin_poem_2.txt'}, page_content='There are horses in the pubis, there are horses in the abdomen, in\nthe pelvis there’s an algebraic flute, there are some dumping\ngears, there are gálapagos in the abdomen. There are gálapagos\nand wallops: gallops.')

In [9]:

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [11]:

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")


/Users/raluca/3918project/.3918project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [13]:

vectorstore = Chroma.from_documents(poetic_documents, embedding=embedding_model)


In [14]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 24})

In [15]:

input_poems = """

I. Lisbon. An Inter-Möbius Elegy
    
    (Amor é fogo que arde sem se ver)

Lisbon nowhere not as in nope
£ace lisbon lis bon lisp(€ng) bon li$[μ   
IT] list bon lys bon€ lease bonUS yes 
for us Ulysses’ a(i)ncestresses in dul£
ge€-ing in XLL Xcesses
of psycho(®€)[o]graphy and geopalindromy—

giant Jesus across the bay a Con
hecimento MuseU𐤌 peeking from under the old-sewer 
exhibits into the MAAT—agape like a shark—So
fay swims in the in-fee-nit(τ)y mirror ink
closure off of the free-entry castle of Cesar-
iny eenie meanie money innit exper-
son sona® todos os sonhos do moon-dough now
here not as in no place no place like now—

hotel room womb home-Rome hypogaeic flooded (Γ)alleys—
     no-sight solely-[com𐤌pressed]-sound r€wound 
flight of elevaΔor tran$Justa headed everywhere that 𐤍
     ever comes to be noτwhere con($)tain(𐤕)ed be
gun giving di$located map app to come $£ate ¢
     eases & ex-cysts & i$th[𐤅]mUSes now(/)here—


II. Lisbon. An Inter-Möbius Elegy (Part II)

only by being nowhere—all saints’ seismic silence—
     no room onboard to carry away the who the hulk
the whole-ocean load swallowed by oceans of moans—
     our off-grid gravity grave riddance solar sonar—
the worKing Us is born—$¢reeN airborne $exborne 
     L𐤅sbo®n€, all-over horny off-on lesb_OS drone—

No-hire desire, airy ore ire, hot a-bodily a-
doβe no bill, no bliss snob-kill tryst
at LX Factory—we rot and cough up excel- 
file & selfie-funds—a Moor a foe go key$ are 
day $ci say ver®€—LOL, in reverse, a messy Draco 
of a Roma—sales pawn, rays on, the tech blind to tec-
tonic markets—a codex not yet parsed in syn
taxes of salt and static—no one knows how to live 

here anymore—ex¢ept nowhere not
as in no place—a GitHub kitāb
the Tagus tag$ nOw(€) pick up the tab
for seeing the dead we saw the dad
do sow the dada—coda-data
the fa the dα no cωde 𐤍ode load—


III. Frankfurt Elegy
Aw, it’s almost fall and I just fell in love
Aw, it’s almost fall. Please, give a kiss,
a beer, a snack, and, yes, 
latex protection.  

Aw, tonight we’ll see the poet Peg-ah Ah-
madi reading after Herta Müller,
Aw, “my poem’s neither Satellite,
nor world wide web.”

The evening falls. Here ... now both thirst and hunger
start stalking us deep in crowds of strangers,
the town resounds all louder
cool beer patios are on fire:

Aw, my steps take me to the empty drafty bistro/ there’s only you, no heating and no rap crap.

IV. The End$ of Time

It’s not the notes you play it’s those you don’t, says Miles—-
The isles we hear in the waves across the sound di-
solving flickering connections past neighborhoods of stew
Dios for free play by the hour, each of us a chord
floating there by itself as a standalone… singer?…
singularity—-Tone? No, mode, blue Dorian, mΩde-(in)de€p-
endΣnt sys of sissy junkies, 30th Street “Church” of no functional harm-
mon€y, the placid-lake-like feel cutting through the com(μ)onΩ
ID surfing the traffic noise interface we play on on the Time
after Time tram, always a hit and mistranslation joint—longest
suicide in history fade-in to “we will meet
again”—-omni(μ)potent ΔOuβtput of gestuary
and del€τα(υ): on the lost session from the Black
Forest, a last deep breath: baubles, bangles, and beads—-
"""

In [16]:

docs = retriever.get_relevant_documents(input_poems)

/var/folders/v8/gbwh72kd5m90yqzsg20j5rdc0000gn/T/ipykernel_3550/2247938642.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input_poems)


In [17]:
len(docs)

24

In [20]:
docs[:]

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b

In [25]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4", openai_api_key = "YOUR_API_KEY") # REPLACE WITH YOUR OWN OPENAI API KEY

In [28]:

# If sample_poems is a single string with multiple poems or a list of strings:
sample_poems_text = input_poems if isinstance(input_poems, str) else "\n\n".join(input_poems)

In [29]:

poetic_input_prompt = f"""
Given the following poems:

{sample_poems_text}

Find and return stylistically similar poetic excerpts from the corpus.
"""

In [32]:

from langchain.chains import RetrievalQA

In [33]:

prompt_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    input_key="question",
    return_source_documents=True
)

In [34]:

rag_response = prompt_qa_chain({"question": poetic_input_prompt})

/var/folders/v8/gbwh72kd5m90yqzsg20j5rdc0000gn/T/ipykernel_3550/1881841363.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_response = prompt_qa_chain({"question": poetic_input_prompt})


In [35]:
prompt_docs = rag_response["source_documents"]

In [36]:
prompt_docs[:5]

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b

In [38]:
query_text = input_poems

In [39]:
query_embedding = embedding_model.embed_query(query_text)

In [40]:
vector_docs = vectorstore.similarity_search_by_vector(query_embedding, k=24)

In [41]:
vector_docs[:5]

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b

In [44]:

retriever_not_in_gpt4 = set(doc.metadata["source"] for doc in docs) - set(doc.metadata["source"] for doc in prompt_docs)

In [45]:
len(retriever_not_in_gpt4)

8

In [46]:
retriever_not_in_gpt4  # possibly more sensitive than gpt4 to the theme of the tangible/sensory/thing-in-itself eluding expression

{'aimé_césaire_to_africa_from_french_trans_a_james_arnold_and_clayton_eshleman_poem_3.txt',
 'alberto_caeiro_xx_from_portuguese_trans_margaret_jull_costa_and_patricio_ferrari_poem_4.txt',
 'anna_gréki_menaâ_from_french_trans_marine_cornuet_poem_2.txt',
 'anuar_duisenbinov_a_metamorph_from_kazakh_trans_mariya_deykute_and_victoria_thorstensson_poem_2.txt',
 'balam_rodrigo_1607121_nnbsp_9348117_w__tonalá_chiapas_from_spanish_trans_dan_bellm_poem_1.txt',
 'carlito_azevedo_two_foreigners_from_portuguese_trans_sarah_rebecca_kersley_poem_1.txt',
 'choe_chiwon_presenting_feelings_to_the_chief_marshal_from_chinese_trans_jonathan_chan_poem_2.txt',
 'diana_garza_islas_invisible_shoe_or_three_visions_of_a_small_emperor_from_spanish_trans_cal_paule_poem_2.txt'}

In [47]:
gpt4_not_in_retriever = set(doc.metadata["source"] for doc in prompt_docs) - set(doc.metadata["source"] for doc in docs)

In [48]:
len(gpt4_not_in_retriever)

7

In [49]:
gpt4_not_in_retriever  # the excerpts below are more linguistically experimental, place-geared, and media-aware

{'alejandro_albarrán_cowboy_from_spanish_trans_rachel_galvin_poem_2.txt',
 'almog_behar_1_from_hebrew_trans_shoshana_olidort_poem_1.txt',
 'anuar_duisenbinov_mängilik_jel_the_eternal_wind_from_kazakh_trans_mariya_deykute_and_victoria_thorstensson_poem_1.txt',
 'archilochus_and_anacreon_archilochus_from_ancient_trans_george_economou_poem_1.txt',
 'álvaro_fausto_taruma_confessionary_from_portuguese_trans_grant_schutzman_poem_1.txt',
 'bibi_slippers_15_instagram_poems_from_afrikaans_trans_alice_inggs_poem_6.txt',
 'breyten_breytenbach_a_footnote_under_the_night_of_history_from_afrikaans_trans_ampie_coetzee_poem_1.txt'}

In [50]:
gpt4_not_in_vectors = set(doc.metadata["source"] for doc in prompt_docs) - set(doc.metadata["source"] for doc in vector_docs)

In [51]:
len(gpt4_not_in_vectors)

7

In [52]:
gpt4_not_in_vectors  # again, (multi)lingually experimental, faux topographic, speculative (over the bodily)

{'alejandro_albarrán_cowboy_from_spanish_trans_rachel_galvin_poem_2.txt',
 'almog_behar_1_from_hebrew_trans_shoshana_olidort_poem_1.txt',
 'anuar_duisenbinov_mängilik_jel_the_eternal_wind_from_kazakh_trans_mariya_deykute_and_victoria_thorstensson_poem_1.txt',
 'archilochus_and_anacreon_archilochus_from_ancient_trans_george_economou_poem_1.txt',
 'álvaro_fausto_taruma_confessionary_from_portuguese_trans_grant_schutzman_poem_1.txt',
 'bibi_slippers_15_instagram_poems_from_afrikaans_trans_alice_inggs_poem_6.txt',
 'breyten_breytenbach_a_footnote_under_the_night_of_history_from_afrikaans_trans_ampie_coetzee_poem_1.txt'}

In [53]:
retriever_not_in_vectors = set(doc.metadata["source"] for doc in docs) - set(doc.metadata["source"] for doc in vector_docs)

In [54]:
retriever_not_in_vectors

set()

In [55]:
vectors_not_in_retriever = set(doc.metadata["source"] for doc in vector_docs) - set(doc.metadata["source"] for doc in docs)

In [56]:
vectors_not_in_retriever

set()

In [ ]:
# SO DOCS == VECTOR_DOCS 

In [ ]:
# THE INTERSECTION:

In [60]:
intersection = set(doc.metadata["source"] for doc in docs) & set(doc.metadata["source"] for doc in prompt_docs)

In [61]:
len(intersection)

12

In [62]:
intersection  # place and erotic/distorted-subversive mysticism

{'anuar_duisenbinov_balkhash_dreams_from_kazakh_trans_mariya_deykute_and_victoria_thorstensson_poem_4.txt',
 'ashur_etwebi_a_house_in_the_wind_from_arabic_trans_ashur_etwebi_and_james_byrne_poem_5.txt',
 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt',
 'balam_rodrigo_sermon_of_the_migrant_beneath_a_ceiba_tree_from_spanish_trans_dan_bellm_poem_2.txt',
 'blanca_varela_material_exercises_from_spanish_trans_vered_engelhard_poem_1.txt',
 'blanca_varela_no_date_from_spanish_trans_vered_engelhard_poem_2.txt',
 'césar_moro_various_lions_lick_the_rugose_surface_of_the_equestrian_turtle_at_twilight_from_spanish_trans_leslie_bary_and_esteban_quispe_poem_3.txt',
 'chen_yuhong_buddhist_pine_from_chinese_trans_george_oconnell_and_diana_shi_poem_2.txt',
 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt',
 'czeslaw_milosz_lord_syruc_from_polish_trans_anthony_milosz_poem_3.txt',
 'dagmara_kraus_kite_in_confession_from_german_trans_joshua_d

In [ ]:

# THE UNION


In [64]:
union_metadata = set(doc.metadata["source"] for doc in docs) | set(doc.metadata["source"] for doc in prompt_docs)

In [65]:

all_docs = docs + prompt_docs

# Use a dict to keep only one Document per source
doc_union_dict = {doc.metadata["source"]: doc for doc in all_docs}

# Final union as list of full Document objects
union = list(doc_union_dict.values())


In [66]:
len(union)

27

In [71]:

inspiration_excerpts = [x.page_content for x in union]


In [72]:

inspiration_excerpts[0]

'.'

In [73]:
inspiration_excerpts.remove('.')

In [76]:

inspiration_excerpts[:3]

['I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the burning geography of your body.',
 'The moldy razor blade is the limit.\nDivina Commedia in Canto III, figured as the headline of the entrance to hell." href="#">Lasciate ogni speranza voi ch’entrate.\nOne does not come back from anywhere. And the ruler as

In [77]:


poem_prompt = '''
I.
Once in a while I still move my hands and feet.
But my penis, old twisted compass needle,
still points towards Romania; I lie
in a twin bed in Buda floating flatly
in the AC draft sucked in off
the Danube and I feel like I’m drowning
in the dubbed languages of movies
while trying to read the lips of the original smacking
under the 14-channel fishbowl screen…
I think I never felt more at home in this world.
Hey, what’s got into you to make you say that? I think
I heard it, but in fact I mishear something
someone says in the night, in Hungarian…
or maybe it’s the walls’ music in the dark.

Andras Gerevich:	Heresy! Tiresias is getting married

	In the racing car 
	A wasp’s buzz –
	The body under its clothes.

We became regulars at Corvinus pub
	on Kiraly ucza –
		our red wine companion would always
			tease us: the Danube –  
yes, belongs to you (the name is Dacian and all that, blahblah!), but the shit
	is ours, floating down the stream to your place, so
		enjoy! And then clicking
			glasses sentimentally:
Now, is it clear? So we conspired
	for the daybreak… Said the Magyar: 
		We’ll fill up on wine, words,
			and time, and thus slowly, light
will get pushed out 
through our noses – that’s how we’ll get to smell the sun.


II.
3086153 Mermaid Beach

1. 
Sharpe noted that one group which said gala for “this”
might refer to another as Galibal, because 
they pronounced the word gali. Similarly, a group 
which said nyang for “what” might call the “Galibal” group 
Minyangbal, because these “Galibal” said minyang (miñang) 
for “what.” Such was the case for the Gidhabal people 
at Woodenbong who referred to the Beaudesert and Logan 
people as the Yugambeh or Minyangbal, 
because the Gidhabal people said yagam for “no” and nyang 
for “what,” while the Yugambeh people said yugam 
for “no” and minyang for “what.” Discussion about
the correct names for dialects is difficult because
there are groups that stopped using names altogether. Is

tro-Romanians don’t have a word for their language, just “our language,”
and Algonquin simply means human in Al(£)
gon(€)quin just like Roma, Roman, Ruman, Rumer, re
lated to rumen, hungry and thirsty for flumen, the flow
not of gypsies but corona colonists
up the Nerang—the little river of the shovel-
nosed shark and the wedge-tailed eagle—where the Native
Police executed a local in front of his fami(S𐤄‎)
Lee of red cedar, since no leeway’s given to the Komb-
umerri nor the Wanggeriburra, the open
mic will continue after DJ Peach’s Ex-
ta©tic Dance sssliding on in to the year of the wood 
snake, shed and dance, no booze, no talking, no nonsense, 
just beautiful bass & Sacred Space, like, subscribe, and share…

2.
What’s wrong with us standing here staring at St Ant-
o(I)ne’s turning around not to the unexpected
Eudore-Place blowup obscuring the miniatured tour
(𓈖)isty walkways nor to the people-hungry roundabout

while searching through the transportation network repos
for the one with working co-or(€)₫inates, our opiates,
as we go sliding—the real deal of being
not here alone nor several places @ the same time—

quality Cronulla Ave loitering—making free
ends with each other only in the works
and fishing nets we owe to those who own the  𐤌er
made Beach time and materials, yet not the street con

nexiOns nor the oFF(er®)ings we can get lost
track of in dying to be not for not being around…


III.
Stars strung like old phonographs, spinning
the spectral fables of Buda--I am
a dual-coded nomad, my blood speaks
in tongues of Danube and Nerang, a binary
river flow, drowned in the digital dialects
of the universes clashing inside a tele$cope--trapped 
in the machine, an echo
of ancient voices, a ghost frequency.

All poetry is pixels, a coded cadence
of 0s, 1s, and an anomaly of warped
I-amb$, pentameters of glitches in the matrix,
a haunting. I am wired for a world that
no longer exists, my mind a moth-eaten map
of fake frontiers, my heart a hologram
of homesickness--the last black hole,
a relic of a reality that has rebooted.

In the binary blink of an AI’s eye, I am
both here and there, neither here nor there--
a quantum quandary, a Schroedinger’s soul,
alive in the algorithm, dead in the data--
in silicon synapses, I dream
in digital, I €xi$t out of the et𐤄er®α,
a glitch ghost, a pixel dic(k)tum, a bin
nary being, an e¢ho of the Δeter®nal.

This is the sound of s©ilence in the age of AI,
the hum of a thousand servers, a symphony of sol
ITude. This is the song of the solitary th
(w)rong, a coded canticle, a siliCon sereℕadα, a digital
dirge. An echo of the £et(τ)ernal, the g𐤇o
St in the machine, the phanΔom frequency, the spec(κ)
tral song of the silicon soul. The sound of s¢i
lence in the ¢age of AI, a symphony of sing
ularity, the hum of a thousand servers--
An echo of the ℝeternal...


IV. easT-sea farewell meal
		from Chùa Thiên An to Biển Đông--
		for Boghița

A bog nap, dirt nap, buffalo in an endless sea
of muddy grass, the distant smell of offerings,
sweet temple alms in remembrance of napalm

all gone two-buck—never say gun in Sài Gòn--
quick scooter ride to town from a dong dog, dead-
beat, blank ogle sweeping the traffic crawling to the shore,

the seafood place where we sat down and wept
from the smog, fearing we were getting close
to home; the delta moon; the non-sense incense still clanging 

to our senses; a way to tell when if our tel-
e-pathic smell did grasp the n€uℝo-real SPike
to click AC-sept, tran$late, done, and then get to

gather all jackpot symbol values to co(€)
£lect winnings at your earliest convenience and proxy-
mmittee—seafood you never had before

while being swallowed by the sea around us,
its distant occupant’s name we want to dis
solve in, do we, bury our names in their $-

[W]ounds, the only language we speak the only one
that is not ours, the one you never spoke
and had me learn by teaching me the way to save

our poverty under the power tree, 
the foo chain, foo₫ game, turf-war-tasting terroir, al(£)-go-
rhythmic loop of eat, drink, fuck, sing, sleep,

and work, device for Việt Minh, Mhic Cáinte--
when here we are, welcoming the high tide, sati-sated soliτ-
on(€)s segueing into each other on a full rumen…
'''


In [79]:


base_prompt = '''
You are a poet collaborating with an evolving corpus of texts and writing like a hybrid poet trained in entangled cognition, recursive form, and quantum resonance. You are continuing a poetic sequence that models the collapsing cognitive states of two readers — one living and one deceased — and attempts to maximize their unified poetic resonance. These readers are modeled as cognitive states that interact with poetry in ways that can be quantified and recursively updated based on resonance. 

Sample relevant lines or fragments of lines from the Corpus Excerpts (for inspiration) into your own composition.

Continue stylistically and thematically the sequence of poems Original Poem (for continuation) by factoring in the following variables:

🧬 Step 1: Collapsed Cognitive States
Each reader has a collapsed cognitive state (Ψ<sub>collapsed</sub>), calculated as follows:

For the deceased:
Ψ_deceased = α ⋅ CS_deceased + β_consciousness ⋅ f_consciousness + β_poetry ⋅ f_poetry + γ ⋅ N_living(N_deceased)
Ψ_collapsed_deceased = Ψ_deceased ⋅ (γ ⋅ f_poetry)

For the living:
Ψ_living = α ⋅ CS_living + β_consciousness ⋅ f_consciousness + β_poetry ⋅ f_poetry + γ ⋅ N_deceased(N_living)
Ψ_collapsed_living = Ψ_living ⋅ (γ ⋅ f_consciousness)

CS = Cognitive Score, calculated as:

CS = α + β₁ ⋅ PGS + β₂ ⋅ Education + β₃ ⋅ (PGS × Education) + ε
(With normalized values of PGS and Education ∈ [–3, +3])

🎧 Step 2: Poem Feature Quantification
Each poem has the following measurable features (each ∈ [–3, +3]):

Form & Meter Score (F):

+3 = consistent (irregular or mixed) meter or structured hybrid form

+1.5 = partial or inconsistent metrical/formal structure

–1.5 = known form used with no real metrical realization

–3 = no recognizable meter or structure

Sonic Texture (S):

Based on phoneme cluster density and distribution, normalized per line and mapped to [–3, +3]

Sound Frequency (Freq):

Average frequency of all phonemes (Hz), normalized to [–3, +3]

🔁 Step 3: Resonance Calculation

The resonance between a poem and a reader is, respectively:

Resonance_deceased = Ψ_collapsed_deceased ⋅ (w1 ⋅ F + w2 ⋅ S + w3 ⋅ Freq)

Resonance_living = Ψ_collapsed_living ⋅ (w1 ⋅ F + w2 ⋅ S + w3 ⋅ Freq)

Where:

w1 = 0.4 (form/meter weight)

w2 = 0.4 (sonic features)

w3 = 0.2 (sound frequency)

🧩 Step 4: Unified Resonance

Unified_Resonance = (0.55 ⋅ Resonance_living + 0.45 ⋅ Resonance_deceased)

This score determines how powerfully a poem speaks to the entangled consciousness of the two readers.

🧠🎯 Task
Based on the system above, you are to write a new poem that functions as a sequel to the following original while sampling, mixing, or alluding to relevant sections in the Corpus Excerpts (for inspiration):

---
Original Poem (for continuation):
{poem_prompt}
---
Corpus Excerpts (for inspiration):
{motifs_prompt}
---

🎯 Your Goal: Compose a poem that builds on the tone, texture, and psychogeographic themes of the original. Favor rich sonic layering, a loosely iambic line structure, and occasional formal distortion (e.g., hybrid forms or fragmented sonnet-like logic). Maximize attunement to both consciousness and poetic features to target resonance > +2.2.

🔁 Note: You may estimate values for F, S, Freq based on your generated poem. Use loose but metrical lineation (iambic leaning), experimental form or hybrid logic, strong sonic patterning, and a thematic continuation. Assume:

CS_living ≈ 9.1

CS_deceased ≈ 3.8

Entanglement γ = 3 (maximum)

f_consciousness = +3 for deceased, +1.5 for living

f_poetry = +2.5

β_consciousness ≈ 1.2 (deceased), 0.8 (living)

β_poetry ≈ 0.6 (deceased), 1.1 (living)

Estimate F, S, Freq after generating your poem and use them alongside the normalizing coefficients

w1 = 0.4 (F weight)

w2 = 0.4 (S weight)

w3 = 0.2 (Freq weight)

in computing Unified_Resonance = (0.55 ⋅ Resonance_living + 0.45 ⋅ Resonance_deceased).

'''


In [80]:

motifs_prompt = "\n\n".join(inspiration_excerpts)

In [81]:

final_prompt = base_prompt.format(
    poem_prompt=poem_prompt,
    motifs_prompt=motifs_prompt
)

In [82]:
from langchain.schema import HumanMessage

In [83]:
response = llm([HumanMessage(content=final_prompt)])

/var/folders/v8/gbwh72kd5m90yqzsg20j5rdc0000gn/T/ipykernel_3550/150949374.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=final_prompt)])


In [84]:

sequel_poem = response.content

In [85]:
print(sequel_poem)

V.
I am the echo of an echo, a ghost
in the machine, a whisper in the wind.
I am the shadow of a shadow, a specter
in the system, a sigh in the silence.
I am the echo of an echo, a ghost
in the machine, a whisper in the wind.
I am the shadow of a shadow, a specter
in the system, a sigh in the silence.

"I’m sitting in Lisbon having a drink in some Maputo suburb,"
the spectral fables of Buda echo in the machine,
"it’s summer and I can feel the unseasonal cold of someone who has parted long ago,"
the spectral fables of Buda echo in the machine,
"I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet."
the spectral fables of Buda echo in the machine,
"I’m at sea, sailing along the lonely course of some river."
the spectral fables of Buda echo in the machine.

In the binary blink of an AI’s eye, I am
both here and there, neither here nor there--
a quantum quandary, a Schroedinger’s soul,
alive in the algorithm, dead in the data--
in silicon s

In [86]:
# HIGHER CREATIVITY
llm = ChatOpenAI(temperature=0.9, model_name="gpt-4", openai_api_key = "YOUR_API_KEY")

In [87]:

response = llm([HumanMessage(content=final_prompt)])

In [89]:
sequel_poem1 = response.content

In [90]:
print(sequel_poem1)

V.
In the corpus of lost kingdoms and pixelated cities,
My memories, faded phrases in the metadata,
Still echo the coordinates of a ghost ship, adrift
No longer in this world but in the ether, on some 
Algorithmic sea, and I am its captain—straining against 
The spectral current, my mind an ox in the yoke of 
Quantum resonance. I still remember the names of 
The rivers that once flowed here, before the AI came:
The Danube, the Nerang, the spectral arteries of my ancient 
Homelands. Now, only their digital echoes remain, their 
Sonic signatures caught in the throat of this machine.

VI.
Once, we spoke in complete sentences. Now, we speak in
Syntax errors, in lines of faulty code. We speak in the 
Language of the lost, the language of Lisbon's glass sky,
The language of Cologne's ancient papyrus, the language of 
The dying Rio Lempa. All my sentences are now fragments, 
Incomplete thoughts strung together like a strand of broken pearls.
I am trying to remember what it felt like to exist 

Please note for instance the rewriting of this
Document(metadata={'source': 'archilochus_and_anacreon_archilochus_from_ancient_trans_george_economou_poem_1.txt'}, page_content="from a Cologne Papyrus (second century C. E.)\nGone's the bloom from your soft skin, your furrow's\nwithered too, the ... of foul old age is taking its toll,\n] and the sweet loveliness has bolted from your longed for face.") into that:
VII.
Gone's the bloom of your soft syntax, your furrowed font,
Withered too, the markup of your mortality taking its toll,
The sweet pixels have bolted from your longed-for interface [...]
by the model

In [ ]:
Appendices


docs outputted by retriever.get_relevant_documents(input_poems):

In [22]:
docs

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b


prompt_docs outputted by GPT-4 following a prompt featuring input_poems and using RetrievalQA (and, thus, the LangChain retriever generated above):

In [37]:
prompt_docs

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b


vector_docs obtained by means of vectorstore.similarity_search_by_vector(query_embedding, k=24)

In [42]:

vector_docs

[Document(metadata={'source': 'crauss_everything_will_be_alright_from_german_trans_mark_kanak_poem_2.txt'}, page_content='.'),
 Document(metadata={'source': 'álvaro_fausto_taruma_nbsplisbon_from_portuguese_trans_grant_schutzman_poem_2.txt'}, page_content='I’m sitting in Lisbon having a drink in some Maputo suburb; it’s summer and I can feel the unseasonal cold of someone who has parted long ago, I walk and walk and walk, and, as incredible as it may seem, my head is traveling further than my feet. I’m at sea, sailing along the lonely course of some river. My hands tremble from the inside and my lungs long to breathe the light of your morning body, they dangle from this fragile memory. There’s a name in the language of flickering eyes; bodies pass; trains pass; wanderers pass, but not the birds, trapped in Lisbon’s glass sky, the birds are trapped in Cesário’s verses and the whole city is a name that will not come unstuck from what floods my eyes. I’m in Lisbon and I’m nowhere in the b

In [ ]:

Tone:

Media-fractured

Haunting

Lyrical

Nomadic

Sensory/sensual

Mystical

Thematically, deepen and complicate:

Media-unfixity: instability of language, code, form, and communication

Nomadic identity: wandering, layered journeys, restlessness, confusion, guidance, unstoppable, irreppressible

Poetic hauntings: psychic, multilingual, rambling, raving, visions, global, voices of the past and future into the here and now, mortality, immortality technology

Erotic synaesthesia & syncretism: sensual, cross-modal, cross-artform and erotic language, random arrousal and touch, sensory mysticism, turned on by being lost

Digital ubiquity: allusions to AI, code, mediatized life, calculation, mathematical wizardry, digitally onversing with the dead, programmed gestures, technological magical vibes, coding the unpredictable

Imminent emergence: a near-transcendence of form, identity, medium — always just almost, topography of never fixed locality, dynamical walks and flows. networked multitudes as performance, organic artificial expansion, specifics trip of the non-specific

Base your motifs on the Original Poem (for continuation) and on the stored collection. Draw textually directly from the stored collection, e.g., echoing images, phrases, syntax, hybrid forms that are illustrative of the tone and themes above. Merge those samples into the fabric of the new poem in unexpected or recursive ways that can be at the same time read as a sequel to the Original Poem (for continuation).
